## 🧼 Wildlife Collision Data Cleaning Notebook

Goal: Clean, structure and enrich the raw wildlife collision dataset for ML model training and analysis.

In [4]:
# -------------------------------------
#  1. Import libraries and load Excel files
# -------------------------------------
import pandas as pd
import glob

# Load all original Excel files (ignore header rows, mark source file)
files = glob.glob("../data/original_excels/*.xlsx")
dfs = []
for file in files:
    df = pd.read_excel(file, skiprows=6)
    df["Source_File"] = file
    dfs.append(df)

# Combine all Excel sheets into one dataframe
all_data = pd.concat(dfs, ignore_index=True)
print(f"✅ Loaded {len(all_data)} rows from {len(files)} files")

✅ Loaded 635851 rows from 10 files


In [5]:
# -------------------------------------
#  2. Save raw combined data
# -------------------------------------
all_data.to_csv("../data/raw_collision_data.csv", index=False)


In [6]:
# -------------------------------------
#  3. Clean column names
# -------------------------------------
all_data.columns = (
    all_data.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("å", "a")
    .str.replace("ä", "a")
    .str.replace("ö", "o")
)

print("🧹 Cleaned column names:")
print(all_data.columns.tolist())

🧹 Cleaned column names:
['DjurId', 'OlycksId', 'Kalla', 'Typ_av_olycka', 'Datum', 'Tid', 'År', 'Manad', 'Dag_pa_aret', 'Unnamed:_9', 'Unnamed:_10', 'Veckodag', 'Lan', 'Unnamed:_13', 'Kommun', 'Viltslag', 'Lat_WGS84', 'Long_WGS84', 'Lat_RT90', 'Long_RT90', 'Kon', 'Årsunge', 'Vad_har_skett_med_viltet', 'Source_File']


In [7]:
# -------------------------------------
# 4. Convert date fields and extract components
# -------------------------------------
# Convert date columns
all_data["Datum"] = pd.to_datetime(all_data["Datum"], errors="coerce")
all_data["Tid"] = pd.to_datetime(all_data["Tid"], errors="coerce", format="%H:%M")

# Extract time-based features
all_data["Manad"] = all_data["Datum"].dt.month
all_data["Veckodag"] = all_data["Datum"].dt.day_name()
all_data["Ar"] = all_data["Datum"].dt.year
all_data["Dag_pa_aret"] = all_data["Datum"].dt.dayofyear

print("📅 Extracted time features")


📅 Extracted time features


In [8]:
# -------------------------------------
# 5. Drop unnecessary columns
# -------------------------------------
cols_to_drop = ["Unnamed:_9", "Unnamed:_10", "Unnamed:_13", "Source_File"]
for col in cols_to_drop:
    if col in all_data.columns:
        all_data = all_data.drop(columns=[col])

print("🗑️ Dropped unused columns")

🗑️ Dropped unused columns


In [11]:
# -------------------------------------
# 6. Translate categorical values to English
# -------------------------------------

# Species
species_translation = {
    "Alg": "Moose", "Radjur": "Roe Deer", "Vildsvin": "Wild Boar",
    "Dovhjort": "Fallow Deer", "Kronhjort": "Red Deer", "Bjorn": "Brown Bear",
    "Lodjur": "Lynx", "Järv": "Wolverine", "Ovriga_djur": "Other", "Okant": "Unknown"
}

# Sex
sex_translation = {"Hane": "Male", "Hona": "Female", "Okant": "Unknown"}

# Juvenile
juvenile_translation = {"Ja": "Yes", "Nej": "No", "Okant": "Unknown"}

# Fate
fate_translation = {
    "Avlivat": "Euthanized", "Dott_pa_olycksplatsen": "Dead at crash site",
    "Ej_patraffat": "Not found", "Bedoms_oskadad": "Assumed unharmed",
    "Patraffat_dott": "Found dead", "Okant": "Unknown"
}

# Apply translations
all_data["Species"] = all_data["Viltslag"].map(species_translation)
all_data["Sex"] = all_data["Kon"].map(sex_translation)
all_data["Juvenile"] = all_data["Årsunge"].map(juvenile_translation)
all_data["Animal_Outcome"] = all_data["Vad_har_skett_med_viltet"].map(fate_translation)

# Drop original Swedish columns
all_data = all_data.drop(columns=["Viltslag", "Kon", "Årsunge", "Vad_har_skett_med_viltet"])

print("🌍 Translated values to English")

🌍 Translated values to English


In [12]:
# -------------------------------------
# 7. Rename columns to English
# -------------------------------------
column_rename = {
    "DjurId": "Animal_ID", "OlycksId": "Collision_ID", "Kalla": "Source",
    "Typ_av_olycka": "Type_of_Collision", "Datum": "Date", "Tid": "Time",
    "Ar": "Year", "Manad": "Month", "Dag_pa_aret": "Day_of_Year", "Veckodag": "Weekday",
    "Lan": "County", "Kommun": "Municipality",
    "Lat_WGS84": "Lat_WGS84", "Long_WGS84": "Long_WGS84",
    "Lat_RT90": "Lat_RT90", "Long_RT90": "Long_RT90"
}

source_translation = {
    "Jagarrapporterad_olycka_(viltolycka.se)": "Hunter-reported collision (viltolycka.se)",
    "Polisrapporterad_olycka_(viltolycka.se)": "Police-reported collision (viltolycka.se)"
}

collision_type_translation = {
    "Vag": "Road", "Jarnvag": "Railway"
}

# Rename and map
all_data = all_data.rename(columns=column_rename)
all_data["Source"] = all_data["Source"].map(source_translation).fillna(all_data["Source"])
all_data["Type_of_Collision"] = all_data["Type_of_Collision"].map(collision_type_translation).fillna(all_data["Type_of_Collision"])

print("✅ Columns renamed and mapped")

✅ Columns renamed and mapped


In [13]:
# -------------------------------------
# 8. Save cleaned dataset
# -------------------------------------
all_data.to_csv("../data/cleaned_data.csv", index=False)
print("💾 Saved cleaned data to cleaned_data.csv")

💾 Saved cleaned data to cleaned_data.csv
